# <font color = 'pink'> Introduction

<font color = 'pink'>This project examines food access inequality or colloquially known as "food deserts" in the southeast neighborhoods of Chicago and the adjacent south suburbs of Dolton, Harvey, Calument City, Riverdale and South Holland.
<font color = 'pink'>Using spatial data on the grocery store locations and community boundaries, the goal is to identify areas that lack nearby grocery stores with quality foods, such as fruits and vegetables.
<font color = 'pink'>A food desert is typically defined as a region where residents have limited access to affordable and nutritrous food within one mile of their homes. "Specifically, the criteria are that at least 500 people or 33% of the population must live more than 1 mile from a large store in an urban area, or more than 10 miles away in a rural area" (USDA, 2020). There are maps that exist on the internet of these food deserts, but I wanted to challenge myself by building my own.

In [27]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import folium

In [28]:
cca_path = Path('community_areas.geojson')
places_path = Path('tl_2024_17_place.shp')
stores_csv = Path('Business_Licenses_20250919.csv')

#load boundaries
if cca_path.exists():
    communities = gpd.read_file(cca_path)
else: 
    communities = gpd.read_file('https://data.cityofchicago.org/resource/igwz-8jzy.geojson')
    communities.to_file(cca_path, driver='GeoJSON')
    
#bounding box
bbox = (-87.75, 41.60, -87.45, 41.76)
communities = communities.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]

#municipal boundaries
municipalities = gpd.read_file(places_path)

In [29]:
#define study area
southeast_names = [
    "South Chicago", "East Side", "South Deering",
    "Hegewisch", "Calumet Heights", "South Shore"
]

# select southeast Chicago neighborhoods
southeast = communities[communities["community"].isin(southeast_names)]

# select south suburbs
south_suburbs = municipalities[municipalities["NAME"].isin(
    ["Dolton", "Harvey", "Calumet City", "Riverdale", "South Holland"]
)]

# project both layers
target_crs = "EPSG:26971"
southeast = southeast.to_crs(target_crs)
south_suburbs = south_suburbs.to_crs(target_crs)

# combine into one study area polygon
from shapely.ops import unary_union
study_poly = unary_union(list(southeast.geometry) + list(south_suburbs.geometry))
study = gpd.GeoDataFrame(geometry=[study_poly], crs=target_crs)


In [30]:
df = pd.read_csv(stores_csv)

df.rename(columns={
    'LONGITUDE': 'lon',
    'LATITUDE': 'lat'},
    inplace=True)

df = df[(df['lon'].between(bbox[0], bbox[2])) &
        (df['lat'].between(bbox[1], bbox[3]))]

#filter by NAICS grocery code
if 'NAICS_CODE' in df.columns:
    df = df[df['NAICS_CODE'].astype(str).str.startswith('445110')]
    

stores = gpd.GeoDataFrame(
    df,
    geometry = gpd.points_from_xy(df['lon'], df['lat']),
    crs = 'EPSG:4326').to_crs(target_crs)

#keep only stores within area
stores = gpd.sjoin(stores, study, op='within').drop(columns=['index_right'])

/cvmfs/cybergis.illinois.edu/software/conda/cybergisx/python3-0.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (9,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
#1 mile grocery access buffer
if len(stores):
    coverage_1mi = gpd.GeoDataFrame(
        geometry=[stores.unary_union.buffer(1609.34)],
        crs=target_crs)
else:
    coverage_1mi = gpd.GeoDataFrame(geometry=[], crs=target_crs)

In [34]:
southeast_4326 = southeast.to_crs(epsg=4326)
south_suburbs_4326 = south_suburbs.to_crs(epsg=4326)
coverage_1mi_4326 = coverage_1mi.to_crs(epsg=4326)
stores_4326 = stores.to_crs(epsg=4326)
m = folium.Map(location=[41.7, -87.6], zoom_start=11, tiles='cartodbpositron')

if len(southeast_4326):
    folium.GeoJson(
        southeast_4326,
        name='Chicago Southeast Side',
        style_function=lambda x: {"color": "blue", "weight": 1.5}
    ).add_to(m)

if len(south_suburbs_4326):
    folium.GeoJson(
        south_suburbs_4326,
        name='South Suburbs',
        style_function=lambda x: {"color": "blue", "weight": 1.5, "dashArray": "5,5"}
    ).add_to(m)

if len(coverage_1mi_4326):
    folium.GeoJson(
        coverage_1mi_4326,
        name='1-Mile Grocery Access',
        style_function=lambda x: {"color": "pink", "fillOpacity": 0.3}
    ).add_to(m)

if len(stores_4326):
    for _, row in stores_4326.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=4,
            color='red',
            fill=True,
            fill_opacity=0.8,
            popup=row.get('DOING BUSINESS AS NAME', 'Grocery Store')
        ).add_to(m)

folium.LayerControl().add_to(m)
m



# <font color = 'pink'> Conclusion
<font color = 'pink'> This analysis highlights the extreme rarity of access to quality and nutritious foods in the Southeast Chicago Area and South Suburbs. A quick Google search to double check my code showed that the closest grocery stores to the Illinois-Indiana border town of Calumet City showed that most grocery stores (based off name alone) were in Indiana. Very interesting. 
<font color = 'pink'> By mapping one mile grocery store buffers, we can see that areas such as South Deering fall outside of this zone, suggesting a higher likelihood of food deserts in that area. We see only one grocery store which is worrisome for the massive communities living in this area. This aligns with known socioeconomic disparities of the region where funding is lacking as well as low income, low car ownership and urban density influence access to healthy food. 
<font color = 'pink'>This map underscores the importance of continual funding to these areas as a result of equitable urban planning. It is very easy to improve public transit in this area. Hopefully, the extended CTA Red Line into Riverdale will incentivize more movement in and out of the city, therefore allowing people to travel for better jobs and allow more money and economic boost into the community.